In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
import numpy as np
import itertools
import dpa_load
import multiprocessing as mp
from datetime import datetime
from itertools import repeat
import split_articles
import numeric_articles
import continue_articles
# we import some functions from the Handelsblatt folder
import sys
sys.path.insert(1, os.getcwd().replace('dpa code', 'Handelsblatt'))
import count_words_mp
import identify_eng_2
import correct_url
import clean_dpa_articles
import clean_dpa_references

In [2]:
# Set the number of cores to use
NUM_CORE = mp.cpu_count()-4

# DPA Data (1991 - 2018)

Deutsche PresseAgentur (DPA) is the Germany's biggest news agency which sells its news reports to the leading German newspapers. We believe that the data set has a high chance to be useful for economic forecasting because DPA produces information that is timely and has a large reach.

We purchased DPA data in November 2019. The corpus consists of **7,539,874** articles from January 1991 to December 2018.

The data set includes news from both dpa-Basisdienstes and dpa-afx Wirtschaftsnachrichten. The former one is the basic news service covering such topics as Economy, Politics, and  Finance. The second one was created in 1999. It specializes in financial news.

## Load the data

First, we read in the data by extracting the following XML elements:

* title - article's title
* text - text of the article
* date - publication date
* ressort - section (Politics vs Economy)
* source/credit - source (dpa vs afx)
* city - which city the news article refers to
* genre - journalistic genre, e.g., chronology, story, table
* wortanzahl - word count
* keywords - keywords associated with an article

In [3]:
# Folder with unpacked articles
#path = r'E:\\Userhome\\jbaer\\dpa_unpacked'

#path = r'G:\\Test\\Results\\dpa Raw Data\\dpa_unpacked'
path = os.getcwd().replace('\\newspaper_data_processing\\dpa code', '') + '\\dpa_unpacked'

folder_list = []

# 2 folders for dpa and dpa-afx 
for fol in [fol for fol in os.listdir(path)]:

    # Within each folder: folders for different years
    for f in [f for f in os.listdir(path + '\\' + fol)]:
        folder_list.append(path + '\\' + fol + '\\' + f)

In [4]:
# Select a path to the folder for storing results
#PATH = r'G:\\Test\\Results'
#os.chdir(PATH)

In [5]:
# Use the 'dpa_load' function to load articles
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    df_list = pool.map(dpa_load.dpa_load, folder_list)
    data = pd.concat(df_list)
    data.reset_index(inplace=True, drop=True)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:47:39.996008


In [6]:
print(len(data))

7539874


In [7]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [8]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,5739189.xml,1,1,1991,pl,dpa,Schalck-Golodkowski,Schalck: Milliardenkredit sicherte Zahlungsfäh...,Berlin,,218,WiPo
1,Tschads Regierung: Bevölkerung soll Waffen abl...,5739191.xml,1,1,1991,pl,dpa,Tschad,Tschads Regierung: Bevölkerung soll Waffen abl...,N'Djamena,,75,WiPo
2,Welajati: Iran bleibt bei einem Krieg am Golf ...,5739193.xml,1,1,1991,pl,dpa,Golfkrise Iran,Welajati: Iran bleibt bei einem Krieg am Golf ...,Teheran,,90,WiPo
3,Bush will offenbar seinen Außenminister erneut...,5739195.xml,1,1,1991,pl,dpa,Golfkrise USA,Bush will offenbar seinen Außenminister erneut...,Washington,,181,WiPo
4,Morgenzusammenfassung Neue Runde diplomatische...,5739199.xml,1,1,1991,pl,dpa,Golfkrise,Morgenzusammenfassung Neue Runde diplomatische...,Washington/Luxemburg,,504,WiPo


In [9]:
#data.to_csv('dpa_raw.csv')

In [3]:
data = pd.read_csv('dpa_raw.csv', encoding = 'utf-8', index_col = 0,  keep_default_na=False,
                   dtype = {'rubrics': 'str', 
                            'source': 'str',
                            'keywords': 'str',
                            'title': 'str',
                            'city': 'str',
                            'genre': 'str',
                            'wordcount': 'str'})

In [4]:
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

In [5]:
data.head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic
0,Schalck: Milliardenkredit sicherte Zahlungsfäh...,5739189.xml,1,1,1991,pl,dpa,Schalck-Golodkowski,Schalck: Milliardenkredit sicherte Zahlungsfäh...,Berlin,,218,WiPo
1,Tschads Regierung: Bevölkerung soll Waffen abl...,5739191.xml,1,1,1991,pl,dpa,Tschad,Tschads Regierung: Bevölkerung soll Waffen abl...,N'Djamena,,75,WiPo
2,Welajati: Iran bleibt bei einem Krieg am Golf ...,5739193.xml,1,1,1991,pl,dpa,Golfkrise Iran,Welajati: Iran bleibt bei einem Krieg am Golf ...,Teheran,,90,WiPo
3,Bush will offenbar seinen Außenminister erneut...,5739195.xml,1,1,1991,pl,dpa,Golfkrise USA,Bush will offenbar seinen Außenminister erneut...,Washington,,181,WiPo
4,Morgenzusammenfassung Neue Runde diplomatische...,5739199.xml,1,1,1991,pl,dpa,Golfkrise,Morgenzusammenfassung Neue Runde diplomatische...,Washington/Luxemburg,,504,WiPo


# Pre-processing

## Light pre-processing

### Remove short articles (<100 words)

Short articles are often incoherent or contain only insiginicant news. For this reason we decided to filter out articles that consist of less than 100 words. 

In [6]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:01:41.576763


In [7]:
# Save the result as a new column "word_count"
data['word_count'] = count_results

In [8]:
# remove articles with less than 100 words
data = data[data['word_count']>=100]
data.reset_index(inplace=True, drop=True)
print(len(data))

5364634


In [9]:
#data.to_csv('dpa_prepro_step1.csv')

### Remove exact duplicates

Some articles are more than once in the corpus. We filter out all duplicates and only keep the articles with the oldest date.

In [10]:
data.drop_duplicates(['texts'], keep = 'first', inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4542316


In [11]:
#data.to_csv('dpa_prepro_step2.csv')

## Filtering

Unfortunately, dpa articles are not as consistently sorted into sections and subsections as articles from other 
news media. Instead, we investigate the most commonly used titles and keywords and remove non-economic articles based on them.

We exclude non-economic and other irrelevant articles based on the following titles

* 1) Londoner Edelmetallpreise: Precious metal prices (without text)
* 2) Tageskalender: List of upcoming events
* 3) (Tabelle), TABELLE: : Tables in text form
* 4) SPORT, Sport (except for the titles that contain TRANSPORT, INTERSPORT, PASSPORT, Sportartikel, news about sportswear manufacturers, sporting goods industry, SPORTARTIKLER, Sportmodelle, or Sportswear): news related to Sports. Beware that news about sports marketing agencies (e.g., Sportfive), sports media websites (e.g., Sportal), and new sports models of car manufacturers might be removed as well. 
* 5) Berichtigung, KORREKTUR (except for the titles that contain Berichtigungsaktien, rectified shares): Article corrections
* 6) Impressum: Dpa contact data
* 7) Testmeldung: Test-articles from dpa
* 8) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, KURSE DREI, KURSE Drei, Kurse drei: Stock charts without text
* 9) DGAP-DD: DGAP reports
* 10) New Yorker Aktien-Schlußkurse: Stock closing prices at the New York stock exchange (articles only occur from 1997 to 2002)
* 11) VERMISCHTES: Miscellaneous with no relation to economics
* 12) Angekündigte US-Quartalszahlen auf einen Blick: Quarterly US figures
* 13) Terminvorschau: Appointment preview
* 14) Aus der Landespolitik: News reports on regional politics. We think they are unlikely to be important for our research question. 
* 15) Die Woche in Berlin, Die Woche in Bonn, Die politische Woche: Announcements of political (and economic) news for next week. The format is very different from other news reports and difficult for the models we use.
* 16) Notizen aus der Politik, NOTIZEN AUS DER POLITIK: Collection of varios short political articles which contentwise  do not seem to be relevant for our research question and are covered only within the limited time period, 2002-2011.
* 17) Presseschau (except for the titles that contain 'zu'): Press reviews that include only headlines of important news reports from various media outlets. We can not analyze the full texts and headlines together because different types of data require different models. Articles with headlines containing both 'Presseschau' and 'zu' are actual press reviews (not just headlines) of a single newspaper on a single topic.
* 18) ÜBERBLICK: Analysten-Umstufungen: Changes in stock ratings (different article format).
* 19) Chronologie, CHRONOLOGIE: Chronology - important dates and events. We remove these articles because for our research question backward-looking articles are arguable not that important.
* 20) dpa-Landesdienst: Regional news unlikely to be important for our research question.
* 21) Die Top-Themen am Aktienmarkt: Short news articles about stock market. They do contain relevant information, but they also have a very different format and they were first issued in 2011. 
* 22) Aktien Asien Schluss.: Quantitative information about the stock market.
* 23) Börsentag auf einen Blick: Articles about the stock market, different format, a lot of quantitative information.
* 24) US-Quartalszahlen vom Vortag: Quarterly stock market figures, tables.

In [12]:
# Filter out non-economic articles based on titles.
fil_titles = '''Londoner Edelmetallpreise|Tageskalender|\(Tabelle\)|SPORT|Sport|Berichtigung|KORREKTUR|Impressum|Testmeldung|Kurse A[^a-z]|Kurse B[^a-z]|Kurse C[^a-z]|Kurse D[^a-z]|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|Kurse/drei|DGAP-DD|New Yorker Aktien-Schlusskurse|VERMISCHTES|Angekündigte US-Quartalszahlen|Terminvorschau|Aus der Landespolitik|Die Woche in Berlin|Die Woche in Bonn|Die politische Woche|Notizen aus der Politik|NOTIZEN AUS DER POLITIK|ÜBERBLICK: Analysten-Umstufungen|TABELLE:|Chronologie|CHRONOLOGIE|dpa-Landesdienst|Die Top-Themen am Aktienmarkt|Aktien Asien Schluss\.|Börsentag auf einen Blick|US-Quartalszahlen vom Vortag'''
titles_exc = '''TRANSPORT|INTERSPORT|PASSPORT|Berichtigungsaktien|Sportartikel|SPORTARTIKLER|Sportmodelle|Sportswear'''
data.drop(data[(data['title'].str.contains(fil_titles, na = False)) & (~data['title'].str.contains(titles_exc, na=False))].index, inplace=True)
# Filter out press reviews that consist of headlines only
data.drop(data[(data['title'].str.contains('Presseschau', na = False)) & (~data['title'].str.contains('zu', na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4147351


We exclude articles based on a title and a text:

* 1) A title contains 'Überblick: ANALYSTEN-EINSTUFUNGEN', and a text contains 'Folgende Investmentbanken haben sich': Changes in stock ratings (different article format).

In [13]:
# Filter out articles based on a title and a text.
data.drop(data[data.title.str.contains('Überblick: ANALYSTEN-EINSTUFUNGEN', na = False) & data.texts.str.contains('Folgende Investmentbanken haben sich', na=False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4144358


We exclude non-economic articles based on the following sections.

* 1) Tabelle: Tables in text form (some articles are still left after the previos step)
* 2) Historisches: News about historical events
* 3) Achtung: Announcemt of upcoming news

In [14]:
# Filter out non-economic articles based on sections.
fil_genres = '''Tabelle|Historisches|Achtung'''
data.drop(data[data['genre'].str.contains(fil_genres, na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

4121293


We exclude non-economic articles based on the following keywords.
* 1) Redaktionshinweis: Editor's notes for Dpa journalists
* 2) DGAP: DGAP reports
* 3) Sport, SPORT, SPO (except for Sportartikel, this section contains articles on sports companies): Sport news (some sports articles are still left after the previos steps)
* 4) Kurse A, Kurse B, Kurse C, Kurse D, KURSE A, KURSE B, KURSE C, Kurse D,
     KURSE DREI, Kurse drei, KURSE Drei, KURSE drei, Kurse Drei, Kurse/drei: Stock charts without text (some articles are  still left after the previos steps)
* 5) Tagesvorschau, Vorschau, VORSCHAU, vorschau: List of titles of upcoming news
* 6) Bilderdienst: Dpa Picture Service
* 7) Geschichte: News related to historical events
* 8) Landespolitik: Regional news irrelevant for economic forecasting

In [15]:
# Filter out non-economic articles based on keywords.
fil_keywords = '''Redaktionshinweis|DGAP|Sport|SPORT|SPO|Kurse A|Kurse B|Kurse C|Kurse D|KURSE A|KURSE B|KURSE C|KURSE DREI|Kurse drei|KURSE Drei|KURSE drei|Kurse Drei|Kurse/drei|Kurse D|Tagesvorschau|Vorschau|vorschau|VORSCHAU|Bilderdienst|Geschichte|Landespolitik'''
keywords_exc = '''Sportartikel'''
data.drop(data[(data['keywords'].str.contains(fil_keywords, na = False)) & (~data['keywords'].str.contains(keywords_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3878655


We exclude articles based on the following bits of text.
* 1) Schalterverkaufskurse: Precios metal prices
* 2) dpa-news.de: News regarding the Dpa website
* 3) Wirtschafts- und Finanztermine, Wirtschafts- und Finanz-Termine, Konjunktur- und Wirtschaftstermine: List of dates when economic data will be published/economic events will take place
* 4) DGAP (except for articles that contain DGAP standing for Deutsche Gesellschaft für Auswärtige Politik): DGAP reports
* 5) Bitte verwenden Sie diese Meldung nicht: Retracted articles
* 6) \( Wiederholung: Repeated articles
* 7) [§] 26 Abs., § 15a WpHG 1, § 15 WpHG, Artikel 19 MAR, article 19 Market Abuse Regulation (MAR): Regulatory news
* 8) Die Pivotpunkte für den Dax-Future: Pivot points for the Dax-Future
* 9) An der Frankfurter Wertpapierbörse wurden, Die Aktien im Dow Jones EuroStoxx 50, Die Aktien im Dow Jones Euro Stoxx 50: Stock charts
* 10) Ihr Ansprechpartner: Redaktion Politik International: List of current political news headlines
* 11) (Achtung - Sonderdisposition): List of articles on a particular topic

In [16]:
fill_text = '''Schalterverkaufskurse:|dpa-news\.de|Wirtschafts- und Finanztermine|Wirtschafts- und Finanz-Termine|DGAP|Bitte verwenden Sie diese Meldung nicht|Konjunktur- und Wirtschaftstermine|\(Wiederholung|[§] 26 Abs\. 1|§ 15a WpHG|§ 15 WpHG|Artikel 19 MAR|article 19 Market Abuse Regulation \(MAR\)|Die Pivotpunkte für den Dax-Future|An der Frankfurter Wertpapierbörse wurden|Die Aktien im Dow Jones EuroStoxx 50|Die Aktien im Dow Jones Euro Stoxx 50|\(Achtung - Sonderdisposition\)'''
text_exc = '''Auswärtige Politik'''
data.drop(data[(data['texts'].str.contains(fill_text, na = False)) & (~data['texts'].str.contains(text_exc, na=False))].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3678700


We want to exclude non-economic articles based on the following two sources.
* 1) dpa-frei: Article corrections
* 2) dpa-wahl: Articles about federal election results

In [17]:
data.drop(data[data['source'].str.contains('dpa-frei|dpa-wahl', na = False)].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3678670


We exclude articles regarding dpa itself.

In [18]:
data.drop(data[data['city'] == 'Die Deutsche Presse-Agentur'].index, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

3678665


In [19]:
#data.to_csv('dpa_prepro_step3.csv')

### Split up articles

Sometimes multiple articles are collected and merged into one entry. For example, articles with the title, keyword, or genre 
'Nachrichtenüberblick' are a collection of the most important articles of the day. Because these smaller articles
can have different sentiments and topics, we separate articles that consist of multiple smaller articles. Articles consisting of multiple smaller articles can be identified with the following words which can appear in titles, keywords, or genres.

- dpa-Nachrichtenüberblick, Nachrichtenüberblick: An overview of news for the upcomming days or news which are a few days old
- Kurznachrichten Wirtschaft: Collection of short economic news
- Analysten-Einstufungen, ANALYSTEN-EINSTUFUNGEN: Analyst stock ratings

We delete one of the articles containing 'Nachrichtenüberblick' in the title because it has too many errors that make it difficult to split the article into paragraphs.

In [20]:
# The article to delete
file_drop = ['8180362.xml']
ind = data[data.file == '8180362.xml']['texts'].index[0]
data[data.file == '8180362.xml']['texts'][ind]

'     Rentenreform verabschiedet\n      BERLIN - Der Weg für die Förderung der privaten Altersvorsorge\nmit knapp 21 Mrd. DM ist frei.  e K\n B h e i r t de. Auch das rot-rot-regierte Mecklenburg-Vorpommern\nstimmte zu. Die SPD-CDU-Koalition von Bremen enthielt sich. In\nMecklenburg-Vorpommern droht nach der überraschenden Zustimmung von\nMinisterpräsident Ringstorff eine Koalitionskrise. Nach PDS-Ansicht\nhat die SPD den Koalitionsvertrag gebrochen und damit den\nRegierungspartner herausgefordert.\n     Mieter erhalten mehr Rechte\n     BERLIN - Die mehr als 20 Mio. Mieter in Deutschland erhalten von\nSeptember an mehr Rechte. Die Grenze für Mieterhöhungen wird gesenkt\nund die Kündigungsfristen werden zu Gunsten der Mieter geändert. Das\nsieht die Mietrechtsreform der Bundesregierung vor, die am Freitag im\nBundesrat die letzte parlamentarische Hürde nahm. Bisher betrugen die\nKündigungsfristen für Mieter un d Vermieter gleichermaßen maximal\nzwölf Monate. Im Interesse beruflicher Mo

In [21]:
data.drop(data[data['file'].isin(file_drop)].index, inplace=True)

While testing the code, we have decided to delete a quantitative part of the following two articles:

In [22]:
ind1 = data[data.file == '8833807.xml']['texts'].index[0]
data[data.file == '8833807.xml']['texts'][ind1]

'     BERLIN(dpa-AFX) - Mit dem Bundeshaushalt 2004 und dem Finanzplan\nbis 2007 bleibt die von der Bundesregierung geplante Neuverschuldung\nvorerst im kritischen Bereich. Sie soll jetzt im kommenden Jahr 28,8\nMilliarden Euro betragen, nachdem Finanzminister Hans Eichel (SPD)\nund die Koalition übereingekommen sind, 2 Milliarden Erlöse aus dem\nVerkauf weiterer Telekom- <DTE.ETR> und Postaktien <DPW.ETR> an die\nKreditanstalt für Wiederaufbau zu erzielen. Diese soll das Paket\nspäter bei guten Kursen an der Börse platzieren.\nFür das laufende Jahr hat Eichel inzwischen eine Neuverschuldung\nvon 35 Milliarden in den Haushaltsentwurf eingestellt, spricht aber\nbereits von einer Verdoppelung der zunächst gesetzlich\nverabschiedeten 18,9 Milliarden Euro. Dies wären etwa 38 Milliarden\nneue Schulden. Die im Herbst erwartete Höhe soll dann in einen\nNachtragshaushalt aufgenommen werden. Andere Fachleute der Koalition\nsprechen schon von rund 40 Milliarden Euro. Die Opposition nennt\nunter 

In [23]:
ind2 = data[data.file == '8833809.xml']['texts'].index[0]
data[data.file == '8833809.xml']['texts'][ind2]

'     BERLIN (dpa-AFX) - Der Entwurf des Bundeshaushalts 2004 sieht\nAusgaben in Höhe von 251,2 Milliarden Euro vor. Das sind 1,2 Prozent\nmehr als für das laufende Jahr gesetzlich geplant, aber 2,3 Prozent\nweniger als das Finanzministerium jetzt für 2003 mit 257 Milliarden\nEuro erwartet. Die neuen Schulden sind mit 28,8 Milliarden Euro\neingeplant.\nDas größte Ausgabevolumen verwaltet das Ressort für Gesundheit und\nSoziales. Die hohe Steigerungsrate im Arbeits- und Wirtschaftsetat\nvon Minister Wolfgang Clement (SPD) wird durch den hohen Zuschuss an\ndie Bundesanstalt für Arbeit von 5,2 Milliarden Euro bestimmt.\n\nDie Übersicht über die Einzelpläne:\nSollEntwurfVeränderung\n20032004zum Vorjahr\n(in Millionen Euro)(Prozent)\n--------------------------------------\nBundespräsidialamt20,4722,96+ 12,2\nBundestag540,73548,93+1,5\nBundesrat17,0617,80+4,3\nBundeskanzleramt1.483,561.488,06+0,3\nAuswärtiges Amt2.229,912.183,40-2,1\nInnen4.014,004.092,58+2,0\nJustiz345,35344,27-0,3\nFinanze

In [24]:
data.at[ind1, 'texts'] = data['texts'][ind1].split(' Hier die Eckwerte')[0]
data.at[ind2, 'texts'] = data['texts'][ind2].split('\n\nDie Übersicht')[0]

In [25]:
s_mult_art = '''dpa-Nachrichtenüberblick|Nachrichtenüberblick|Kurznachrichten Wirtschaft|Analysten-Einstufungen|ANALYSTEN-EINSTUFUNGEN'''
mult_art = data[data['title'].str.contains(s_mult_art, na = False)]
mult_art = mult_art.append(data[data['keywords'].str.contains(s_mult_art, na = False)])
mult_art = mult_art.append(data[data['genre'].str.contains(s_mult_art, na = False)])
mult_art.drop_duplicates(['texts'], keep = 'first', inplace=True)
mult_art.reset_index(inplace=True, drop=True)

In [26]:
# delete 'mult_art' from the original data
#data.drop(data[data['title'].str.contains(s_mult_art, na = False)].index, inplace=True)
#data.drop(data[data['keywords'].str.contains(s_mult_art, na = False)].index, inplace=True)
#data.drop(data[data['genre'].str.contains(s_mult_art, na = False)].index, inplace=True)
#data.reset_index(inplace=True, drop=True)

In [26]:
# calculate chunck size 
chunk_size = int(mult_art.shape[0]/NUM_CORE)

# split data into chunks 
chunks = [mult_art.iloc[mult_art.index[i:i + chunk_size]] for 
          i in range(0, mult_art.shape[0], chunk_size)]

In [1214]:
# split up articles into smaller articles and append the resulting new articles 
# to the corpus
from datetime import datetime
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    results = pool.map(split_articles.split_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

results = pd.concat(results)
print(len(results))

0:01:01.360557
925146


In [1215]:
results.reset_index(inplace=True, drop=True) # reset the index of the DataFrame

The separated articles consist of fewer words than the articles from which they originally stemmed. Therefore, we count the number of words of the new articles with the count_words_mp function from before and filter out articles with less than 100 words.

In [1216]:
# count the number of words for the separated articles and filter out articles with less
# than 100 words
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    count_results = pool.map(count_words_mp.count_words_mp, [text for text in results['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)
results['word_count'] = count_results

0:00:05.448739


In total, we have almost 90000 split articles longer than 100 words.

In [1229]:
len(results[(results['word_count']>=100)])

89156

Right now, we concentrate on the articles from the WiPo section. We have around 15000 of those.

In [1230]:
results[(results['word_count']>=100) & (results.topic == 'WiPo')].head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic,word_count,dpa_tag
72,RWE geht in Telekommunikation eigene Wege - Al...,3660743.xml,7,9,1995,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 07.09.19...,Düsseldorf/EssenBonnFrankfurt/Main,,347,WiPo,102,no
2062,Verhandlungen über Kredite für Fokker - Druck ...,3799057.xml,25,1,1996,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 25.01.19...,Den Haag/StuttgartEindhoven/FürthMünchenBonnFr...,,482,WiPo,102,no
2628,Opel will Schadenersatz von VW - VW will Klage...,3841869.xml,8,3,1996,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 08.03.19...,Rüsselsheim/WolfsburgBerlinFrankfurt/Main,,266,WiPo,100,no
2965,BMW-Konzern will beim Gewinn wieder Gas geben ...,3866307.xml,2,4,1996,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 02.04.19...,MünchenHannoverStuttgartAmsterdamFrankfurt/Main,,421,WiPo,102,no
3209,Tarifeinigung in der Druckindustrie Frankfurt/...,3880821.xml,19,4,1996,pl,dpa,Nachrichtenüberblick dpa Politik,dpa-Nachrichtenüberblick POLITIK 19.04.1996 - ...,New YorkTel AvivKairoDannenbergNeu Delhi,,351,WiPo,122,no


Some of them are split incorrectly. One way to find false negatives is to find articles with more than one dpa reference.

In [1218]:
startTime = datetime.now()
import find_strings_re

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dpa_tag_results = pool.map(find_strings_re.find_strings_re, [text for text in results['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:00:04.122106


Here are the articles we will deal with now. Mariia's first example is 23154, Jasper's - 25216.

In [1231]:
results['dpa_tag'] = dpa_tag_results
results[(results['dpa_tag']=='yes') & (results.topic == 'WiPo') & (results['word_count']>=100)].head()

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic,word_count,dpa_tag
15360,Verkauf von Rolls-Royce an VW perfekt - Kaufpr...,4642369.xml,3,7,1998,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 03.07.19...,London/WolfsburgDüsseldorfFrankfurt/Main,,413,WiPo,101,yes
23154,Verband: Kein Ende des Streits mit Telekom übe...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,113,yes
25216,Auch der Maschinenbau boomt in die Qualifizier...,3051967.xml,29,2,2000,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,Nachrichtenüberblick/dpa/Wirtschaft/ dpa-Nachr...,GüterslohDüsseldorfFrankfurt/MainMünchenWiesba...,,680,WiPo,177,yes
25227,Koch-Weser offiziell bei IWF-Exekutivrat angem...,3052360.xml,1,3,2000,pl,dpa,Nachrichtenüberblick dpa Politik,dpa-Nachrichtenüberblick POLITIK 01.03.2000 - ...,WashingtonJerusalemHamburgBerlin,,435,WiPo,156,yes
26692,CDU-Spendenaffäre: Verfahren gegen Kohl noch o...,3095157.xml,6,5,2000,pl,dpa,Nachrichtenüberblick dpa Politik,dpa-Nachrichtenüberblick POLITIK 06.05.2000 - ...,Belfast/LondonManilaFurnas/AzorenStuttgartBonn...,,546,WiPo,108,yes


In [1234]:
# The way the article is split now:
results[results.file == '5324041.xml']

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic,word_count,dpa_tag
23153,Allianz mit Pimco-Übernahme in Weltliga der Ve...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,91,no
23154,Verband: Kein Ende des Streits mit Telekom übe...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,113,yes
23155,C&N Touristic mit kräftigem Umsatzplus - für G...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,101,no
23156,Duisenberg heizt Erwartungen an: Zinserhöhung ...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,88,no
23157,Heizöl-Käufer warten ab - Preise weit höher al...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,94,no
23158,Aktienmarkt hält sich über 5 500 Punkten - All...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,75,no
23159,EZB: Euro gestiegen = Frankfurt/Main (dpa) - D...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,39,no


In [1235]:
# Find the unsplit article
mult_art[mult_art.file == '5324041.xml']

,texts,file,day,month,year,rubrics,source,keywords,title,city,genre,wordcount,topic,word_count
3827,Allianz mit Pimco-Übernahme in Weltliga der...,5324041.xml,1,11,1999,wi,dpa,Nachrichtenüberblick dpa Wirtschaft,dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...,MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...,,587,WiPo,601


In [1238]:
# Call it test (row['texts'] in the function)
test = mult_art['texts'][3827]

In [1239]:
# Info on the article
mult_art.iloc[3827]

texts            Allianz mit Pimco-Übernahme in Weltliga der...
file                                                5324041.xml
day                                                           1
month                                                        11
year                                                       1999
rubrics                                                      wi
source                                                      dpa
keywords                    Nachrichtenüberblick dpa Wirtschaft
title         dpa-Nachrichtenüberblick WIRTSCHAFT - 01.11.19...
city          MünchenKölnAgenturTozeur/TunisFrankfurt/MainHa...
genre                                                          
wordcount                                                   587
topic                                                      WiPo
word_count                                                  601
Name: 3827, dtype: object

Tests

In [1240]:
'WEITERE MELDUNGEN:\n' in test

False

In [1241]:
'W I R T S C H A F T\n' in test

False

In [1242]:
len(re.findall('\n\s+?\(Sperrfrist.+\)', test))

0

In [1243]:
re.findall(r'(?:\s{2,})[/A-ZÄÖÜß]{4,}\b', test)

[]

In [1244]:
re.findall(r'(?:\n)[A-ZÄÖÜß-]{5,}\b.{0,}(?<![a-z])\n', test)

[]

In [1245]:
re.findall(r'\(dpa.*?\)', test)

['(dpa)', '(dpa)', '(dpa)', '(dpa)', '(dpa)', '(dpa)', '(dpa)', '(dpa)']

In [1246]:
dpa_ref = re.findall(r'\(dpa.*?\)', test)

In [1247]:
txt = test
txt = txt.strip().replace("\n", ' ').replace("\t", ' ')

In [1248]:
#txt

In [1249]:
len(re.findall(r'\./\s', txt))

0

In [1250]:
re.findall(r'((?:^|(?:\s{2}))(?:[\'A-ZÄÖÜß0-9:\-\(]).+?(?:\s{2}(?!\-|<|[a-z]| [a-z])|$))', txt)

['Allianz mit Pimco-Übernahme in Weltliga der Vermögensverwalter =  ',
 '  München (dpa) - Der Versicherungskonzern Allianz steigt mit der milliardenschweren Übernahme der US-Gesellschaft Pimco in die Weltliga der Vermögensverwalter auf. «Das ist für uns ein Riesenschritt nach vorne beim Aufbau unseres neuen Kerngeschäftsfelds», sagte der Vorstandsvorsitzende der Allianz AG (München), Henning Schulte-Noelle, am Montag. Die Allianz werde für eine Mehrheitsbeteiligung von knapp 70 Prozent rund 3,3 Milliarden Dollar (6,1 Mrd DM/3,12 Mrd Euro) bezahlen. Mit dem Zukauf wird die Allianz künftig gemeinsam mit Pimco als sechstgrößter Asset Manager der Welt ein Vermögen von rund rund 1 200 Milliarden DM verwalten.  ',
 '  Köln (dpa) - Im Streit zwischen der Deutschen Telekom und ihren Konkurrenten um das Einziehen der Telefongebühren zeichnet sich kein Ende ab. Die Wettbewerber seien bereit, die Kosten für das Inkasso der Telekom anteilig zu übernehmen, sagte der Geschäftsführer des Verbandes d

In [1251]:
mult_art_e = re.findall(r'((?:^|(?:\s{2}))(?:[\'A-ZÄÖÜß0-9:\-\(]).+?(?:\s{2}(?!\-|<|[a-z]| [a-z])|$))', txt)

In [1252]:
'Kurznachrichten/Wirtschaft'  in test

False

In [1253]:
len(re.findall(r'(?:^[ ]*|(?<=\.\n)[ ]*)(?:[A-ZÄÖÜßa-z\n])[^.]+?(?:=)', test))

7

In [1254]:
len(mult_art_e) > len(dpa_ref) and len(dpa_ref) > 1 

True

In [1255]:
re.findall(r'(?:^|(?<=\.\n{1}\s)|(?<=\.\n{2})|(?<=»\n{1}\s))[^\.]+?(?=\n[\s]*?[A-ZÄÖÜß][A-ZÄÖÜa-zäöüß /]+ - )',test)

[]

In [1256]:
 len([r.strip() for r in re.findall(r'(?:^|(?<=\.\n{1}\s)|(?<=\.\n{2})|(?<=»\n{1}\s))[^\.]+?(?=\n[\s]*?[A-ZÄÖÜß][A-ZÄÖÜa-zäöüß /]+ - )',test) if r.strip() != '']) > 0

False

In [1257]:
re.findall(r'(?:^|(?<=\.\s{2})|(?<=\.»\s{2})|(?<=(?<!dpa)\)\s{2})|(?<=Maschinenbauers\s{2}))[\S\s]+?(?=\(dpa(?!\-Grafik).+?)', txt)

['Allianz mit Pimco-Übernahme in Weltliga der Vermögensverwalter =     München ',
 'Verband: Kein Ende des Streits mit Telekom über Rechnungen in Sicht =      Köln ',
 '  C&N Touristic mit kräftigem Umsatzplus - für Gebühren in Reisebüros=     Tozeur/Tunis ',
 ' Duisenberg heizt Erwartungen an: Zinserhöhung gilt nahezu als sicher=     Frankfurt/Main ',
 '   Heizöl-Käufer warten ab - Preise weit höher als vor einem Jahr =     Hamburg ',
 ' Aktienmarkt hält sich über 5 500 Punkten - Allianz im Mittelpunkt =     Frankfurt/Main ',
 ' EZB: Euro gestiegen =     Frankfurt/Main ']

In [1258]:
headlines = re.findall(r'(?:^|(?<=\.\s{2})|(?<=\.»\s{2})|(?<=(?<!dpa)\)\s{2})|(?<=Maschinenbauers\s{2}))[\S\s]+?(?=\(dpa.+?)', txt)

In [1259]:
len(headlines) < len(mult_art_e)/2

False

In [1260]:
re.findall(r'(?:^|(?<=\s{2}))[^\(][\S\s]+?(?=\(dpa(?!\-Grafik).+?)', txt)

['Allianz mit Pimco-Übernahme in Weltliga der Vermögensverwalter =     München ',
 'Verband: Kein Ende des Streits mit Telekom über Rechnungen in Sicht =      Köln ',
 '  C&N Touristic mit kräftigem Umsatzplus - für Gebühren in Reisebüros=     Tozeur/Tunis ',
 ' Duisenberg heizt Erwartungen an: Zinserhöhung gilt nahezu als sicher=     Frankfurt/Main ',
 '   Heizöl-Käufer warten ab - Preise weit höher als vor einem Jahr =     Hamburg ',
 ' Aktienmarkt hält sich über 5 500 Punkten - Allianz im Mittelpunkt =     Frankfurt/Main ',
 ' EZB: Euro gestiegen =     Frankfurt/Main ']

We will continue from here when we deal with all the articles that are split incorrectly.

In [28]:
results['word_count'] = count_results
results = results[results['word_count']>=100]

In [29]:
# append separated articles to corpus
data = data.append(results)
data = data.sort_values(['year', 'month', 'day'], ascending=[True, True, True]) # sort the data in chronological order
data.reset_index(inplace=True, drop=True) # reset the index of the DataFrame
print(len(data))

2031724


In [30]:
#data.to_csv('dpa_prepro_step4.csv')

### Identify and Delete English Articles

In [37]:
# Delete all English articles from the data  
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    eng_results = pool.map(identify_eng_2.identify_eng_2, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

1:48:39.715753


In [48]:
data['language'] = eng_results
data = data[data.language==0]
data.reset_index(inplace=True, drop=True)
print(len(data))

2064333


### Identify articles that predominantly consist of numbers

Articles that consist predominately of numbers sometimes carry little sentiment. Filtering out all numbers only helps a little, because the resulting texts are often grammatical nonsensical. To get a better understanding of how these articles look we identify all articles that consist of more than 80% of numbers.

In [31]:
# use the 'numeric_articles' function to identify economic articles with a high share of numbers in them
inputs = zip(data['texts'], data['word_count'], itertools.repeat(0.80))

startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    numeric_list = pool.starmap(numeric_articles.numeric_articles, inputs)
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:01:23.476436


In [32]:
numeric = data[numeric_list]

In [33]:
# inspect example article with high share of numbers
numeric['texts'].iloc[0]

'FLENSBURG (dpa-AFX) - Der langjährige Renner VW-Golf <VOW.ETR> ist auch 1999 an der Spitze der Auto-Neuzulassungen in Deutschland geblieben. Den 356.000 Golf-Exemplaren - einschließlich seiner Abwandlungen Vento und Bora - folgten erneut rund 230.000 Opel-Astra <GM.NYS>. Das geht aus einer Statistik der Kraftfahrt-Bundesamts (KBA) in Flensburg hervor, die am Dienstag veröffentlicht wurde. Insgesamt kamen 1999 knapp über 3,8 Millionen fabrikneue Personenwagen in den Straßenverkehr. 1998 waren es 3,73 Millionen.      Auf den dritten Platz liegen mit 143.000 Erstanmeldungen die Fahrzeugtypen der Dreier-Reihe von BMW <BME.ETR>, die den ebenfalls seit Jahren mit führenden VW-Passat (136.500) auf den vierten Rang verdrängten. Neuling in der 1999er Tabellenspitze der 25 Wagentypen - von insgesamt über 70 - sind der Ford-Focus <F.NYS> auf Platz fünf mit 113.000 Neuzulassungen, der VW-Lupo (16./61.000), der Peugeot 206 (22./44.000) sowie der Opel-Zafira (25./38.500). Auf Erfolgskurs ist auch d

In [34]:
numeric['texts'].iloc[1]

'Operative Kosten                        40.616     48.002    18,2% Vertriebs- und allg.                    25.892     36.359    40,4% Verwaltungskosten Zentrale Verwaltungskosten              16.114     18.884    17,2% Nicht liquiditätswirksame                    0      5.393 Personalkosten aus der Gewährung von Aktienoptionen Abschreibungen und Amortisierung       122.614    119.848    -2,3% ------------------------------------------------------------------ --- Betriebsaufwand                        205.236    228.486    11,3% ------------------------------------------------------------------ --- ------------------------------------------------------------------ --- Betriebsergebnis (-verlust)            -21.193    -21.268     0,4%  ------------------------------------------------------------------ --- Zinsaufwand                            -55.901    -31.046   -44,5% Anderer Aufwand                         -1.298     -1.500    15,6% --------------------------------------------------

In [35]:
len(numeric)

5412

### Merge continuations of articles

Some dpa-afx articles are split into multiple entries marked by the word 'Fortsetzung' at the beginning of the texts of following entries.

In [38]:
data['texts'].iloc[1]

'FRANKFURT (dpa-AFX) - Der deutsche Aktienmarkt hat am Montag für wenige Augenblicke seinen Rekordschluss vom Juli 1998 überschritten. Mit einem Höchststand von 6.188,68 lag der Dax über dem Tageschluss vom 21. Juli 1998, als der Index 6.186 Punkte erreichte. Der Dax <DAX.ETR> schloss den Handel am ersten Wochentag bei 6.142,19 Zählern und damit um 0,38% oder 23,02 Punkte fester ab. Der Nebenwerteindex M-Dax <MDAX.ETR> gab dagegen auf 3.990,00 Punkte oder um 1,43% nach, und der Neue Markt-Index Nemax 50 <NMKX.ETR> schloss bei 4.610,66 Zählern (-0,58%).      Spätestens auf dem Niveau des All-Time-High bei ungefähr 6.200 Punkten werde der Index auf einen massiven Widerstand stoßen, heißt es am Montag in einem Research-Report von der Nürnberger Schmidt Bank. Michael Schubert, Analyst der Bankgesellschaft Berlin, sagte in einem Gespräch mit dpa-AFX, derzeit sei der einzig negative Faktor, dass die Börse nicht von der Breite getragen werde, sondern nur von wenigen Titeln. Wer jetzt noch nic

In [39]:
data['texts'].iloc[3]

'(Fortsetzung) - Der Tagesgewinner SAP <SAP.ETR> übernahm bereits am Vormittag die Führung bei den Gewinnern ein und schloss den Handel mit einem Kurszuwachs von 6,27% auf 475,20 Euro ab. Händler waren sich einig darüber, dass SAP weiteres Wachstumpotenzial haben. Ende vergangener Woche habe es Kaufempfehlungen für das Papier gegeben, sagte ein Händler. Die ABN Amro Bank hatte ihr Kursziel mit 600 Euro festgesetzt.      Den Wert umgaben zudem Gerüchte über eine bevorstehende Allianz mit einem Weltkonzern, hieß es. Ein anderer Händler hielt den aktuellen SAP-Kurs noch immer für zu billig. Wachstumswerte stünden weiter in der Gunst der Anleger. Bei SAP hätten Image und Aktienkurs unter Problemen in den USA gelitten. Das werde nach Erwartung des Händlers aber spätestens mit den Geschäftszahlen des ersten Quartals 2000 überwunden sein. Mit dem Ende der Angst vor dem Jahr-2000-Problem würde der Kurs weiter steigen.     Ihren Kurshöhenflug fortsetzen konnten zudem die Aktien von Siemens <SIE

In [40]:
data['texts'].iloc[7]

'(Fortsetzung) - Überraschend fest tendierten am Montag die Autowerte. Volkswagen <VOW.ETR> gewannen 1,6% auf 49,35 Euro. Vielleicht sehen wir jetzt tatsächlich eine Branchen-Rotation zur Autobranche hin, sagte ein Händler der DG-Bank. Die nahe Zukunft des Wertes stehe oder falle mit der 50 Euro Marke. Auch BMW <BMW.ETR> (28,10 Euro/+1,04%) und DaimlerChrysler <DCX.ETR> (67,25 Euro/+0,67%)  verbuchten Kurszuwächse. Alle drei deutschen Hersteller verbuchten einem Pressebericht zufolge im Gesamtjahr 1999 steigende Absatzzahlen in den USA.       Die Verliererliste führten am Montag Lufthansa <LHA.ETR> (22,30 Euro/-3,88%) und Veba <VEB.ETR> (45,47 Euro/-3,87%) an. Unter Gewinnmitnahmen litt laut Händlern der Kurs der T-Aktie <DTE.ETR>: Der Titel verlor im Montagshandel 1,31% auf 59,30 Euro. Wäre die Telekom mit dem Indextrend im Wert gestiegen, hätte der Dax den Rekordstand vom Juli 1998 eingestellt, rechnete ein Händler am Nachmittag vor./mr/fs'

We identify which entries belong together and merge them to one article

In [46]:
# Divide data into roughly equal sized chunks where articles from one day only fall under the same chunk
data_cont = data[data['topic'] == 'afx']
dates = data_cont.groupby(['year', 'month', 'day'])
dates = list(dates.groups)
dates = np.array_split(dates, NUM_CORE)

meta_data = data_cont.loc[:, data_cont.columns != 'texts']

chunks = [pd.concat([data_cont[(data_cont['year'] == t[0]) & (data_cont['month'] == t[1]) & (data_cont['day'] == t[2])] for t in tup]) for tup in dates]
chunks = [[chunk, meta_data] for chunk in chunks]

In [48]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    continue_results = pool.map(continue_articles.continue_articles, chunks) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:14:56.636717


In [49]:
data.drop(list(itertools.chain(*[tup[0] for tup in continue_results])), inplace=True)

In [ ]:
continue_articles = pd.concat([tup[1] for tup in continue_results])
print(len(continue_articles))

In [50]:
data = data.append(continue_articles)
data.reset_index(inplace=True, drop=True)
print(len(data))

1974426


In [51]:
#data.to_csv('dpa_prepro_step6.csv')

## Remove URLs

In [52]:
startTime = datetime.now()

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    url_corrected = pool.map(correct_url.correct_url, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:06:33.551105


In [53]:
data['texts'] = url_corrected

## Remove dpa references

We remove dpa references (e.g. NEW YORK (dpa) - ...) from each article.

In [54]:
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    dpa_ref_removed = pool.map(clean_dpa_references.clean_dpa_references, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:15:15.393896


In [55]:
data['texts'] = dpa_ref_removed

In [66]:
#data.to_csv('dpa_prepro_step7.csv')

# Correct spelling

### Umlauts

Older articles (1991 - 2000) from the section 'Kommentar' (Commentary) are often missing umlauts and correct capitalization. To
fix these two issues, we use the notebook Umlauts_fix written in Python 2 and the notebook Truecasing written in Python 3.

In [6]:
umlauts = ['ä', 'ö', 'ü', 'ß', 'Ä', 'Ö', 'Ü']
umlauts_replace = ['ae', 'oe', 'ue', 'ss', 'AE', 'OE', 'UE']

In [7]:
dpa_umlauts_fix = data[(data.texts.str.contains('|'.join(umlauts_replace))) & (~data.texts.str.contains('|'.join(umlauts))) & (data.year<2001)]

In [9]:
dpa_umlauts_fix['texts'].iloc[0]

'Ad-hoc announcement sent by DGAP. The sender is solely responsible for the contents of this announcement.  Ad-hoc Mitteilung Nach @ 15 WpHG  WizCom Technologies Ltd. (WizCom) <WZM.FSE> (Neuer Markt:WZM,WKN:915 856) veroeffentlicht das Ergebnis fuer das am 31. Dezember 1999 endende Geschaeftsjahr 20. Maerz 2000, Jerusalem, Israel - Der Umsatz belief sich im Jahr 1999 auf US$ Mio. 11,613. Der Vorjahreswert lag bei US$ Mio. 15,799. Der Umsatz im ersten bzw. zweiten Halbjahr 1999 betrug US$ Mio. 4,046 bzw. US$ Mio. 7,567. Das Unternehmen sieht den Umsatzrueckgang als voruebergehend an und vor allem im zweiten Halbjahr begruendet, weil das neue Produkt, der QuickLink-Pen, erst mit einigen Monaten Verzoegerung auf den Markt gebracht werden konnte. Im 4. Quartal 1999 konnte WizCom den QuickLink Pen in den USA erfolgreich auf den Markt bringen. Im 1. Quartal 2000 vermarktet WizCom den QuickLink Pen in weiteren Laendern, unter anderem Australien, Grossbritannien, Deutschland und Frankreich. De

In [ ]:
dpa_umlauts_fix.to_csv('dpa_umlauts_fix.csv', encoding='utf-8-sig', sep = ';')

In [ ]:
dpa_umlauts_fixed = pd.read_csv('dpa_umlauts_fixed.csv', encoding = 'utf-8', sep=';')

### Truecasing

In [56]:
dpa_cases_fix = data[data.texts.str.contains('^(?!.*[A-Z])')]

In [57]:
dpa_cases_fix['texts'].iloc[0]

'new york (vwd) - enttaeuschend verlief das geschaeft am mittwoch, dem ersten handelstag im neuen jahr, an der new yorker aktienboerse. die zunaechst gesehenen leichten gewinne konnten nur bis in das fruehe nachmittagsgeschaeft behauptet werden. in den letzten 2-1/2 geschaeftststunden gerieten die kurse in die minuszone und wall street schloss auf breiter front schwaecher. der dow-jones-index fuer 30 industriewerte gab um 23,02 auf 2.610,64 punkte nach. auch die uebrigen marktbestimmenden indizes gerieten in die minuszone. bei einem umsatz von 126,28 (114,13) millionen aktien standen die kursverlierer den -gewinnern im verhaeltnis von rund neun zu sieben gegenueber. verantwortlich fuer die schwaeche waren wiederauflebende befuerchtungen ueber eine anhaltende rezessionsphase. nachdem sogar das weisse haus jetzt von einer rezessionaeren entwicklung spricht, hielten sich die meisten anleger mit ihren engagements zurueck, wodurch der vorherrschende abgabedruck ausreichte, um die kurse in d

In [62]:
dpa_cases_fix.to_csv('dpa_case_fix.csv', encoding='utf-8-sig', sep = ';')

In [63]:
dpa_cases_fixed = pd.read_csv('dpa_cases_fixed.csv', encoding = 'utf-8', sep=';')

In [64]:
data.loc[dpa_cases_fixed.index, 'texts'] = dpa_cases_fixed

In [65]:
# fixed version
data['texts'].iloc[0]

'New York( Vwd)- Enttaeuschend verlief das Geschaeft am Mittwoch, dem ersten Handelstag im neuen Jahr, an der New Yorker Aktienboerse. Die Zunaechst Gesehenen leichten Gewinne konnten nur bis in das Fruehe Nachmittagsgeschaeft behauptet werden. In den letzten 2-1/2 Geschaeftststunden gerieten die Kurse in die Minuszone und Wall Street schloss auf breiter Front Schwaecher. Der Dow-Jones-Index Fuer 30 Industriewerte gab um 23,02 auf 2.610,64 Punkte nach. Auch die Uebrigen Marktbestimmenden Indizes gerieten in die Minuszone. Bei einem Umsatz von 126,28( 114,13) Millionen Aktien standen die Kursverlierer den -Gewinnern im Verhaeltnis von rund neun zu sieben Gegenueber. Verantwortlich Fuer die Schwaeche waren Wiederauflebende Befuerchtungen Ueber eine anhaltende Rezessionsphase. Nachdem sogar das Weisse Haus jetzt von einer Rezessionaeren Entwicklung spricht, hielten sich die meisten Anleger mit ihren Engagements Zurueck, wodurch der vorherrschende Abgabedruck ausreichte, um die Kurse in di

## Fixing tokens containing a number and a word

In quite a few cases, a number and a word are erroneously merged into a single token. Splitting these tokens into two tokens helps us to deal with the following problems:

(see Handelsblatt notebook)

In [58]:
startTime = datetime.now()

import split_number_word

if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    split_corrected = pool.map(split_number_word.split_number_word, [text for text in data['texts']]) 
    pool.close()
    pool.join()

print(datetime.now()-startTime)

0:02:13.158789


In [59]:
data['texts'] = split_corrected

In [60]:
#data.to_csv('dpa_prepro_step8.csv')

# Delete Fuzzy Duplicates

In [61]:
types_keep = ['russische Aktienmarkt', 'europäischen Börsen', 'Deutsche Börse' ,'Europäische Zentralbank',
'Moskauer Aktienmarkt', 'deutsche Aktienmarkt', 'deutschen Aktienmarkt', 'Folgende Investmentbanken',
'deutsche Rentenmarkt', 'amerikanischen Treasury Bonds', 'IRW-PRESS', 'Deutsche Bank', 'Ausgewählte Analysten-Einstufungen',
'Deutsche Staatsanleihen', 'Der japanische Aktienmarkt']

exceptions = ['NO_EXCEPTIONS']

In [63]:
# import a function that outputs the indices of duplicates 
import fuzzy_duplicates
delete_indices = []
startTime = datetime.now() 
for year in list(set(data['year'])):
    data_input = data[(data['year'] == year)]
    for month in list(set(data_input[data_input['year'] == year]['month'])): # old: list(set(data))
            # Prepare inputs
            inputs_year = []
            inputs_month = []
            inputs_month_year = []
            inputs_year.append(year)
            inputs_month.append(month)
            inputs_month_year.append(data_input[(data_input['year'] == year) & (data_input['month'] == month)][["month", "year", "texts"]])

            #from itertools import repeat
            inputs = list(zip(inputs_year, inputs_month, inputs_month_year))
            from datetime import datetime
            if __name__ == "__main__":
                pool = mp.Pool(NUM_CORE)
                # apply function to all combinations of month-year in parallel
                delete_intermediate = pool.starmap(fuzzy_duplicates.fuzzy_duplicates, zip(inputs, repeat(types_keep), repeat(exceptions)))
                delete_indices = delete_indices + delete_intermediate # create one list of indices
                pool.close()
                pool.join()  
    print(year)
print(datetime.now()-startTime)

1991.0
1992.0
1993.0
1994.0
1995.0
1996.0
1997.0
1998.0
1999.0
2000.0
2001.0
2002.0
2003.0
2004.0
2005.0
2006.0
2007.0
2008.0
2009.0
2010.0
2011.0
2012.0
2013.0
2014.0
2015.0
2016.0
2017.0
2018.0
6:58:11.847691


In [64]:
delete_indices = list(set([item for sublist in delete_indices for item in sublist]))

In [65]:
data.drop(delete_indices, inplace=True)
data.reset_index(inplace=True, drop=True)
print(len(data))

1580324


In [66]:
#data.to_csv('dpa_prepro_step9.csv')

# Clean articles

Dpa articles include some unnecessary text passages like inquiry notes or references to webpages. We decided to clean the 
affected articles from these text passages to make the sentiment classification easier for our model(s).

We remove the following terms and sections from the texts:
* 1) stock symbols
* 2) additional metadata in the text meant for the author
* 3) references to previos articles
* 4) references to dpa and dpa-AFX webpage
* 5) uncorrected original article on which a correction is based on
* 6) inquiry notes
* 7) reference to english article on which some articles are based on
* 8) date of the article
* 9) references for aditional information (phone numbers, webpages etc.)
* 10) references to sender
* 11) references to Debitos
* 12) references to issuer
* 13) reference to authors
* 14) references to summary of article 

In [67]:
startTime = datetime.now()
if __name__ == "__main__":
    pool = mp.Pool(NUM_CORE)
    cleaned_articles = pool.map(clean_dpa_articles.clean_dpa_articles, [text for text in data['texts']]) 
    pool.close()
    pool.join()
    
print(datetime.now()-startTime)

0:58:52.745164


In [68]:
data['texts'] = cleaned_articles

In [69]:
#data.to_csv('dpa_prepro_final.csv')